In [1]:
pip install numpy pandas scikit-learn nltk rank-bm25 gensim matplotlib tqdm faiss-cpu


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Only run if packages not installed
!pip install numpy pandas scikit-learn nltk rank_bm25


Defaulting to user installation because normal site-packages is not writeable


In [3]:
from nltk.corpus import stopwords

# Get English stopwords
stop_words = set(stopwords.words("english"))
print(stop_words)  # Prints the list of stopwords


{'no', 'will', 'same', "they'd", 'between', 'into', 'only', 'again', 'further', 'had', 'theirs', 'doing', 'other', 'than', 'their', "hasn't", "won't", 'your', 'such', "weren't", 'haven', 'herself', "i'd", "you'd", "wouldn't", 'below', "we'd", "it'll", 'did', 'nor', 'there', 'doesn', 'whom', 'hers', 'itself', "he'll", 'yourselves', 'they', "you've", "we've", 'when', 'on', 'been', 'was', 'after', 'ours', 'isn', 'ourselves', 'both', 'at', 'weren', 'which', 'why', "mustn't", "it's", "you're", "isn't", 'own', 'ain', "shan't", 'over', "mightn't", 'an', "i've", 'y', "she's", 'our', 'o', 'if', 'what', 'd', "aren't", 'then', 'the', "wasn't", 'while', 'against', "she'll", 'more', 'during', 'themselves', 'didn', 'how', 'for', 'under', 'won', "i'll", 'so', 'but', 'are', 'mightn', "they've", 'aren', "he's", 'you', 'yourself', "haven't", 'in', "she'd", 'shouldn', 'by', 'some', 'who', 'yours', 'with', 'm', "that'll", 'it', 'now', 'to', 'very', "we're", 'and', 'i', 'his', 'him', "you'll", 'few', 'my',

In [4]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Initialize stemmer
ps = PorterStemmer()

# Load English stopwords
stop_words = set(stopwords.words("english"))
print("Number of stopwords:", len(stop_words))
print("Sample stopwords:", list(stop_words)[:20])


Number of stopwords: 198
Sample stopwords: ['no', 'will', 'same', "they'd", 'between', 'into', 'only', 'again', 'further', 'had', 'theirs', 'doing', 'other', 'than', 'their', "hasn't", "won't", 'your', 'such', "weren't"]


In [5]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from rank_bm25 import BM25Okapi

# Download NLTK stopwords quietly (won't print messages)
nltk.download('stopwords', quiet=True)

# Initialize stemmer and stopwords
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Example usage
text = "This is a sample sentence showing how to remove stopwords and stem words."

tokens = text.lower().split()
processed_tokens = [ps.stem(w) for w in tokens if w not in stop_words]

print("Original tokens:", tokens)
print("Processed tokens:", processed_tokens)


Original tokens: ['this', 'is', 'a', 'sample', 'sentence', 'showing', 'how', 'to', 'remove', 'stopwords', 'and', 'stem', 'words.']
Processed tokens: ['sampl', 'sentenc', 'show', 'remov', 'stopword', 'stem', 'words.']


In [6]:
def load_documents(path="data/"):
    docs = []
    ids = []

    for filename in os.listdir(path):
        if filename.endswith(".txt"):
            with open(os.path.join(path, filename), "r", encoding="utf8") as f:
                docs.append(f.read())
                ids.append(filename)

    df = pd.DataFrame({"doc_id": ids, "text": docs})
    return df

# Load the dataset
df = load_documents()
df.head()


,doc_id,text


In [7]:
documents = [
    "Information retrieval retrieves relevant documents.",
    "TF-IDF is a weighting scheme used in IR.",
    "The vector space model is commonly applied.",
    "Python is used in machine learning and IR."
]


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)


In [10]:
similarity_matrix = cosine_similarity(tfidf_matrix)
print(similarity_matrix)


[[1.         0.         0.         0.        ]
 [0.         1.         0.064266   0.36226405]
 [0.         0.064266   1.         0.064266  ]
 [0.         0.36226405 0.064266   1.        ]]


In [11]:
from collections import defaultdict

def build_inverted_index(docs):
    index = defaultdict(set)
    for i, doc in enumerate(docs):
        for token in doc.split():
            index[token].add(i)
    return index


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Create documents
documents = [
    "Information retrieval retrieves relevant documents.",
    "TF-IDF is a weighting scheme used in IR.",
    "The vector space model is commonly applied.",
    "Python is used in machine learning and IR."
]

# Step 2: TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

# Step 3: Cosine similarity
similarity_matrix = cosine_similarity(tfidf_matrix)

print(similarity_matrix)


[[1.         0.         0.         0.        ]
 [0.         1.         0.064266   0.36226405]
 [0.         0.064266   1.         0.064266  ]
 [0.         0.36226405 0.064266   1.        ]]


In [13]:
def preprocess(text):
    text = text.lower()  # lowercase
    text = re.sub(r"[^a-z\s]", " ", text)  # remove punctuation/numbers
    tokens = text.split()  # tokenize
    tokens = [t for t in tokens if t not in stop_words]  # remove stopwords
    tokens = [ps.stem(t) for t in tokens]  # stemming
    return " ".join(tokens)

# Apply to all documents
df["clean_text"] = df["text"].apply(preprocess)
df.head()


,doc_id,text,clean_text


In [14]:
def retrieve_tfidf(query, vectorizer, matrix, df, top_k=5):
    q_clean = preprocess(query)
    q_vec = vectorizer.transform([q_clean])  # vectorizer is already fitted
    scores = cosine_similarity(q_vec, matrix)[0]
    top_idx = scores.argsort()[::-1][:top_k]
    return df.iloc[top_idx][["doc_id", "text"]]


In [15]:
def preprocess(text):
    text = str(text).lower()
    text = re.sub(r'\W', ' ', text)
    words = [w for w in text.split() if w not in stop_words]
    return ' '.join(words)


In [16]:
df['tokens'] = df['text'].apply(lambda x: preprocess(x).split())


In [17]:
print(df.shape)          # Should show number of rows
print(df['tokens'].head())  # Should show list of tokens per document


(0, 4)
Series([], Name: tokens, dtype: float64)


In [18]:
# Convert all entries to strings first
df['text'] = df['text'].astype(str)

# Remove rows where text is empty after stripping
df = df[df['text'].str.strip() != '']

# Preprocess and tokenize
df['tokens'] = df['text'].apply(lambda x: preprocess(x).split())

# Keep only rows with non-empty token lists
df = df[df['tokens'].map(len) > 0]


In [19]:
# Remove rows where tokens list is empty
df = df[df['text'].str.strip() != '']  # remove empty text
df['tokens'] = df['text'].apply(lambda x: preprocess(x).split())
df = df[df['tokens'].map(len) > 0]  # keep only rows with non-empty token lists



In [2]:
# --------------------------------------------
# Robust BM25 notebook cell (copy-paste and run)
# --------------------------------------------
import os
import re
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from rank_bm25 import BM25Okapi

# Ensure stopwords are available
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))

# Preprocess function
def preprocess(text):
    text = str(text).lower()
    # keep letters and spaces only
    text = re.sub(r'[^a-z\s]+', ' ', text)
    tokens = [w for w in text.split() if w and (w not in stop_words)]
    return ' '.join(tokens), tokens

# Document loader - reads .txt files from "data/" folder
def load_documents(path="data/"):
    docs = []
    names = []
    if not os.path.exists(path):
        print(f"Warning: path '{path}' does not exist.")
        return pd.DataFrame({"doc_id": [], "text": []})
    for filename in sorted(os.listdir(path)):
        if filename.lower().endswith(".txt"):
            full = os.path.join(path, filename)
            try:
                with open(full, "r", encoding="utf8") as f:
                    txt = f.read()
            except Exception as e:
                # try with latin-1 as a fallback
                try:
                    with open(full, "r", encoding="latin-1") as f:
                        txt = f.read()
                except Exception as e2:
                    print(f"Could not read {filename}: {e2}")
                    continue
            docs.append(txt)
            names.append(filename)
    return pd.DataFrame({"doc_id": names, "text": docs})

# 1) Load documents
df = load_documents("data/")  # change path if needed

# 2) Quick diagnostics before cleaning
print(f"Raw files found: {len(df)}")

# 3) Clean, preprocess and tokenise
if len(df) > 0:
    df['text'] = df['text'].astype(str)
    # remove purely empty or whitespace docs
    df = df[df['text'].str.strip() != ''].copy()
    # apply preprocess (returns cleaned string and tokens)
    cleaned = df['text'].apply(preprocess)
    df['clean_text'] = cleaned.apply(lambda x: x[0])
    df['tokens'] = cleaned.apply(lambda x: x[1])
    # keep only docs with at least 1 token
    df = df[df['tokens'].map(len) > 0].reset_index(drop=True)

print(f"Documents after cleaning/tokenization: {len(df)}")
if len(df) > 0:
    # print basic token-length stats and a small sample
    print("Token length stats: min, median, max =", df['tokens'].map(len).min(),
          int(df['tokens'].map(len).median()), df['tokens'].map(len).max())
    print("\nSample doc ids and token counts:")
    print(df[['doc_id']].assign(token_count=df['tokens'].map(len)).head(5).to_string(index=False))
else:
    print("No valid documents found after preprocessing.")

# 4) If corpus is empty, optionally create a tiny demo corpus so you can continue testing
if len(df) == 0:
    print("\nBuilding demo corpus because no valid documents were found.")
    demo_texts = [
        "Machine learning is a field of artificial intelligence that uses statistical techniques to give computer systems the ability to learn from data.",
        "Information retrieval is the process of obtaining information system resources relevant to an information need from a collection of information resources.",
        "Deep learning is a subset of machine learning concerned with algorithms inspired by the structure and function of the brain called artificial neural networks."
    ]
    demo_ids = [f"demo_{i+1}.txt" for i in range(len(demo_texts))]
    df = pd.DataFrame({"doc_id": demo_ids, "text": demo_texts})
    cleaned = df['text'].apply(preprocess)
    df['clean_text'] = cleaned.apply(lambda x: x[0])
    df['tokens'] = cleaned.apply(lambda x: x[1])
    print("Demo corpus created with", len(df), "documents.")

# 5) Build BM25 index safely
corpus_tokens = df['tokens'].tolist()
if len(corpus_tokens) == 0:
    raise ValueError("Corpus token list is empty. Check your data folder and preprocessing steps.")
else:
    bm25 = BM25Okapi(corpus_tokens)
    print("BM25 index built. Corpus size:", len(corpus_tokens))

# 6) Retrieval function (returns doc_id, text, score)
def retrieve_bm25_with_scores(query, bm25, df, top_k=5):
    q_clean, q_tokens = preprocess(query)
    if len(q_tokens) == 0:
        print("Warning: query has no tokens after preprocessing. Try a different query.")
        return pd.DataFrame(columns=["doc_id","text","score"])
    scores = bm25.get_scores(q_tokens)
    top_idx = np.argsort(scores)[::-1][:top_k]
    results = df.iloc[top_idx].copy()
    results = results.assign(score = scores[top_idx])
    return results[["doc_id","text","score"]]

# 7) Test query
query = "machine learning algorithms"
results = retrieve_bm25_with_scores(query, bm25, df, top_k=5)
print("\nTop results for query:", query)
display(results)


Raw files found: 0
Documents after cleaning/tokenization: 0
No valid documents found after preprocessing.

Building demo corpus because no valid documents were found.
Demo corpus created with 3 documents.
BM25 index built. Corpus size: 3

Top results for query: machine learning algorithms


,doc_id,text,score
2,demo_3.txt,Deep learning is a subset of machine learning ...,0.743683
0,demo_1.txt,Machine learning is a field of artificial inte...,0.210340
1,demo_2.txt,Information retrieval is the process of obtain...,0.000000


In [22]:
# Convert all entries to strings first
df['text'] = df['text'].astype(str)

# Remove rows where text is empty after stripping
df = df[df['text'].str.strip() != '']

# Preprocess and tokenize
df['tokens'] = df['text'].apply(lambda x: preprocess(x).split())

# Keep only rows with non-empty token lists
df = df[df['tokens'].map(len) > 0]


In [5]:
import os

docs_path = r"C:\projects\ir-system\data"

for file in os.listdir(docs_path):
    if file.endswith(".txt") or file.endswith(".txt.txt"):
        full_path = os.path.join(docs_path, file)
        print(f"\n--- {file} ---")
        with open(full_path, "r", encoding="utf8") as f:
            content = f.read()
            print("Length:", len(content))
            print("Preview:", repr(content[:200]))



--- file1.txt.txt ---
Length: 0
Preview: ''

--- file2.txt.txt ---
Length: 0
Preview: ''


In [1]:
import os
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from rank_bm25 import BM25Okapi

# FIX YOUR PATH HERE
docs_path = r"C:\projects\ir-system\data"

documents = []

# Load .txt files
for filename in os.listdir(docs_path):
    if filename.endswith(".txt"):
        with open(os.path.join(docs_path, filename), "r", encoding="utf8") as f:
            text = f.read().strip()
            documents.append({"doc_id": filename, "text": text})

df = pd.DataFrame(documents)

print("Loaded documents:", len(df))
print(df.head())


Loaded documents: 2
          doc_id text
0  file1.txt.txt     
1  file2.txt.txt     


In [2]:
df = df[df["text"].str.strip() != ""]
print("Non-empty documents:", len(df))


Non-empty documents: 0


In [4]:
print("Loaded documents =", len(df))
print(df[["doc_id", "text"]].head())

print("\nTokenized docs:")
for i, row in df.iterrows():
    print(row["doc_id"], "→", row["tokens"])


Loaded documents = 0
Empty DataFrame
Columns: [doc_id, text]
Index: []

Tokenized docs:


In [6]:
import os
docs_path = r"C:\projects\ir-system\data"

print("FILES FOUND:", os.listdir(docs_path))

print("\n--- DOCUMENT CONTENT CHECK ---")
for filename in os.listdir(docs_path):
    if filename.endswith(".txt") or filename.endswith(".txt.txt"):
        full = os.path.join(docs_path, filename)
        with open(full, "r", encoding="utf8") as f:
            content = f.read()
            print(f"\nFILE: {filename}")
            print("Length of file:", len(content))
            print("Preview:", repr(content[:200]))


FILES FOUND: ['file1.txt.txt', 'file2.txt.txt']

--- DOCUMENT CONTENT CHECK ---

FILE: file1.txt.txt
Length of file: 0
Preview: ''

FILE: file2.txt.txt
Length of file: 0
Preview: ''


In [10]:
import pandas as pd
from nltk.tokenize import word_tokenize
from rank_bm25 import BM25Okapi
import nltk
nltk.download("punkt")

# ------------------------------
# DEMO documents
# ------------------------------
documents = [
    {"doc_id": "doc1", "text": "Machine learning is a field of AI that uses data to train models."},
    {"doc_id": "doc2", "text": "Information retrieval systems help users find relevant documents efficiently."},
    {"doc_id": "doc3", "text": "Deep learning is a subset of machine learning using neural networks."}
]

# Create DataFrame
df = pd.DataFrame(documents)
print(df)


  doc_id                                               text
0   doc1  Machine learning is a field of AI that uses da...
1   doc2  Information retrieval systems help users find ...
2   doc3  Deep learning is a subset of machine learning ...


[nltk_data] Downloading package punkt to C:\Users\SYED ZAIN UL
[nltk_data]     AIDEEN/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
df["tokens"] = df["text"].apply(lambda x: word_tokenize(x.lower()))
df = df[df["tokens"].map(len) > 0]
print(df)


  doc_id                                               text  \
0   doc1  Machine learning is a field of AI that uses da...   
1   doc2  Information retrieval systems help users find ...   
2   doc3  Deep learning is a subset of machine learning ...   

                                              tokens  
0  [machine, learning, is, a, field, of, ai, that...  
1  [information, retrieval, systems, help, users,...  
2  [deep, learning, is, a, subset, of, machine, l...  


In [12]:
tokenized_docs = df["tokens"].tolist()
bm25 = BM25Okapi(tokenized_docs)
print("BM25 index built successfully!")


BM25 index built successfully!


In [13]:
def retrieve_bm25(query, bm25, df, top_k=5):
    q_tokens = word_tokenize(query.lower())
    scores = bm25.get_scores(q_tokens)
    top_idx = scores.argsort()[::-1][:top_k]
    return df.iloc[top_idx][["doc_id", "text"]]

query = "machine learning algorithms"
top_docs = retrieve_bm25(query, bm25, df)
top_docs


,doc_id,text
2,doc3,Deep learning is a subset of machine learning ...
0,doc1,Machine learning is a field of AI that uses da...
1,doc2,Information retrieval systems help users find ...


In [14]:
def retrieve_bm25(query, bm25, df, top_k=5):
    q_clean = preprocess(query)
    q_tokens = q_clean.split()
    scores = bm25.get_scores(q_tokens)
    top_idx = np.argsort(scores)[::-1][:top_k]
    return df.iloc[top_idx][["doc_id", "text"]]


In [16]:
# -----------------------------
# Step 0: Imports
# -----------------------------
import os
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from rank_bm25 import BM25Okapi

nltk.download("punkt")  # only downloads if missing

# -----------------------------
# Step 1: Load documents
# -----------------------------
docs_path = r"C:\projects\ir-system\data"  # change if your folder is elsewhere

documents = []

if os.path.exists(docs_path):
    files = os.listdir(docs_path)
    for filename in files:
        if filename.endswith(".txt"):
            with open(os.path.join(docs_path, filename), "r", encoding="utf8") as f:
                text = f.read().strip()
                if text:
                    documents.append({"doc_id": filename, "text": text})

# -----------------------------
# Step 2: Fallback demo corpus
# -----------------------------
if len(documents) == 0:
    print("No valid .txt files found. Using demo documents...")
    documents = [
        {"doc_id": "doc1", "text": "Machine learning is a field of AI that uses data to train models."},
        {"doc_id": "doc2", "text": "Information retrieval systems help users find relevant documents efficiently."},
        {"doc_id": "doc3", "text": "Deep learning is a subset of machine learning using neural networks."}
    ]

# Create DataFrame
df = pd.DataFrame(documents)
print(f"Loaded {len(df)} documents")

# -----------------------------
# Step 3: Tokenize
# -----------------------------
df["tokens"] = df["text"].apply(lambda x: word_tokenize(x.lower()))
df = df[df["tokens"].map(len) > 0]
print(f"Docs with tokens: {len(df)}")

# -----------------------------
# Step 4: Build BM25 index
# -----------------------------
tokenized_docs = df["tokens"].tolist()
bm25 = BM25Okapi(tokenized_docs)
print("BM25 index built successfully!")

# -----------------------------
# Step 5: Define retrieval function
# -----------------------------
def retrieve_bm25(query, bm25, df, top_k=5):
    q_tokens = word_tokenize(query.lower())
    scores = bm25.get_scores(q_tokens)
    top_idx = scores.argsort()[::-1][:top_k]
    return df.iloc[top_idx][["doc_id", "text"]]

# -----------------------------
# Step 6: Run example query
# -----------------------------
query = "machine learning algorithms"
top_docs = retrieve_bm25(query, bm25, df)
top_docs


No valid .txt files found. Using demo documents...
Loaded 3 documents
Docs with tokens: 3
BM25 index built successfully!


[nltk_data] Downloading package punkt to C:\Users\SYED ZAIN UL
[nltk_data]     AIDEEN/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,doc_id,text
2,doc3,Deep learning is a subset of machine learning ...
0,doc1,Machine learning is a field of AI that uses da...
1,doc2,Information retrieval systems help users find ...


In [ ]:
from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(df['tokens'].tolist())

In [24]:
def precision_at_k(retrieved_docs, relevant_docs, k=5):
    retrieved_k = retrieved_docs[:k]
    return len(set(retrieved_k) & set(relevant_docs)) / k


In [ ]:
query = input("Enter your search query: ")
print("Top documents (TF-IDF):")
display(retrieve_tfidf(query, vectorizer, tfidf_matrix, df))

print("Top documents (BM25):")
display(retrieve_bm25(query, bm25, df))


In [ ]:
# Local Information Retrieval System

## Installation
pip install -r requirements.txt

## Run
1. Open Jupyter Notebook
2. Run all cells
3. Use the search demo cell to query documents
